In [7]:

# -*- coding: utf-8 -*-
"""


@author: Ammar.Abasi
"""

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.io import loadmat, savemat

from keras.models import Sequential
from keras.layers import Dense
from tqdm import tqdm
from keras.callbacks import ModelCheckpoint
import pickle 
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pandas as pd
import os
from tensorflow.keras.optimizers import Adam
import csv
from pprint import pprint
import glob
import re 
import scipy.io
import h5py
import pickle
import random
import math
import warnings

# Define a filter to match the specific part of the warning message
warning_filter = "Casting complex values to real discards the imaginary part"

# Suppress warnings matching the filter
warnings.filterwarnings("ignore", message=warning_filter)
loss_object = tf.keras.losses.MeanSquaredError()
def fgsm(model, input_instance, label):
    eps =0.1
    tensor_input_instance = tf.convert_to_tensor(input_instance, dtype=tf.float32)
    adv_x = input_instance
    count=10
    for idx in range(0,len(label) ):#
      
        #print("idx",idx)
        with tf.GradientTape() as tape:
            tmp_label = label[idx]
            tape.watch(tensor_input_instance)
            prediction = model(tensor_input_instance)
            loss = loss_object(tmp_label, prediction)
            gradient = tape.gradient(loss, tensor_input_instance)
            signed_grad = tf.sign(gradient)
            adv_x = adv_x + eps * signed_grad

    return adv_x


def robust_aggregation_with_adaptive_thresholds_and_outlier_detection(subservers_models):
    aggregated_weights = [np.zeros_like(w) for w in subservers_models[0].get_weights()]

    update_magnitudes = []  # To store magnitudes of model updates

    for subserver_model in subservers_models:
        model_weights = subserver_model.get_weights()
        update_magnitude = [np.linalg.norm(w) for w in model_weights]
        update_magnitudes.append(update_magnitude)

    # Calculate the adaptive threshold based on the update magnitude distribution
    threshold = np.percentile(update_magnitudes, 95)  # Adjust percentile as needed

    for i, subserver_model in enumerate(subservers_models):
        model_weights = subserver_model.get_weights()
        update_magnitude = update_magnitudes[i]

        # Check if all update magnitudes are below the threshold
        if np.all(update_magnitude <= threshold):
            # Apply the update to the aggregated model
            for j, w in enumerate(model_weights):
                aggregated_weights[j] += w

    return aggregated_weights


# Define the CNN model
def create_model(In_train,n_beams):
    
    in_shp = list(In_train.shape[1:])    
    act_func = 'relu'
    model = Sequential()
    model.add(Dense(100, input_dim=in_shp[0], activation=act_func))
    model.add(Dense(100, activation=act_func))
    model.add(Dense(100, activation=act_func))
    model.add(Dense(100, activation=act_func))
    model.add(Dense(n_beams, activation=act_func))
    model.compile(loss='loss', optimizer='rmsprop', metrics=['mean_squared_error'])
    return model

def train(In_train, Out_train, In_test, Out_test,
          nb_epoch,loss_fn,n_beams,device,learning_rate,batch_size):
    
    in_shp = list(In_train.shape[1:])
    
    AP_models = []    
    act_func = 'relu'
    device_optimizer = Adam(learning_rate=learning_rate)
    model = Sequential()
    model.add(Dense(100, input_dim=in_shp[0], activation=act_func))
    model.add(Dense(100, activation=act_func))
    model.add(Dense(100, activation=act_func))
    model.add(Dense(100, activation=act_func))
    model.add(Dense(n_beams, activation=act_func))
    model.compile(loss=loss_fn, optimizer='rmsprop', metrics=['mean_squared_error'])
    
    history = model.fit(In_train,
                                        Out_train[:, device*n_beams:(device+1)*n_beams],
                                        batch_size=batch_size,
                                        epochs=nb_epoch,
                                        verbose=0,
                                        validation_data=(In_test, Out_test[:,device*n_beams:(device+1)*n_beams]))
    #filehandler = open('history.pkl', 'wb') 
    #pickle.dump(history.history, filehandler)
    #filehandler.close()

    AP_models.append(model)
    return AP_models



def hierarchical_federated_learning(num_subservers, num_devices, num_global_rounds, num_local_rounds, num_epochs_per_round, learning_rate,n_beams,batch_size,In_train,In_test,Out_train,Out_test,DL_size_ratio):

    number_sample_per_device= round(len(In_train)/num_subservers/ num_devices)
    print("In_train=",len(In_train))
    print("num_subservers=",num_subservers)
    print("num_devices=",num_devices)
    print("number_sample_per_device=",number_sample_per_device)
  

       

    # Initialize sub-server models
    subservers_models = [create_model(In_train,n_beams) for _ in range(num_subservers)]

    # Initialize the root model
    root_model = create_model(In_train,n_beams)
    
    # Create and open the CSV file for writing

    for global_round in range(num_global_rounds):
        losses=[];
        print("global_round =", global_round)
        
        root_model_weights = root_model.get_weights()
        for subserver_model in subservers_models:
            subserver_model.set_weights(root_model_weights)
    
        for subserver in range(num_subservers):

            # Local training loop
            for local_round in range(num_local_rounds):
                print("local_round =", local_round)
                AP_models = []
                device_models = [create_model(In_train,n_beams) for _ in range(num_devices)]
                for device in range(num_devices):
                    device_model = device_models[device]
                    sample_indices = np.random.choice(len(In_train), size=number_sample_per_device+(global_round+5), replace=False)
                    device_data = In_train[sample_indices]
                    device_labels = Out_train[sample_indices]
                    print("device =", device)
                    print(device_data.shape)
                    nb_epoch=10
                    loss_fn='mean_squared_error'

                    # Update device model with sub-server model weights
                    device_model.set_weights(subserver_model.get_weights())
                    AP_models=train(device_data, device_labels, In_test, Out_test, nb_epoch,loss_fn,n_beams,device,learning_rate,batch_size)
                 
                    if device % 2 == 1 and global_round>=1:
                        In_test_adv = fgsm(device_model, device_data, device_labels[:, subserver*n_beams:(subserver+1)*n_beams])
                        AP_models=train(In_test_adv, device_labels, In_test, Out_test, nb_epoch+global_round,loss_fn,n_beams,device,learning_rate,batch_size)
                        
                    else:
                        AP_models=train(device_data, device_labels, In_test, Out_test, nb_epoch+global_round,loss_fn,n_beams,device,learning_rate,batch_size)
                   
                    
                    beams_predicted=AP_models[0].predict( In_test, batch_size=10, verbose=0)

                    #DL_Result['TX'+str(id+1)+'Pred_Beams']=beams_predicted
                    #DL_Result['TX'+str(id+1)+'Opt_Beams']=Out_test[:,id*n_beams:(id+1)*n_beams]
                    
                    loss = mean_squared_error(Out_test[:,device*n_beams:(device+1)*n_beams],beams_predicted)
                    losses.append(loss)
           
                        
                        
                      
                # Perform model aggregation at the sub-server
                subserver_weights = subserver_model.get_weights()
                subserver_weights_sum = [np.zeros_like(w) for w in subserver_weights]
                for device_model in device_models:
                    device_weights = device_model.get_weights()
                    subserver_weights_sum = [subserver_weights_sum[i] + device_weights[i] for i in range(len(subserver_weights_sum))]
                subserver_weights_mean = [w / num_devices for w in subserver_weights_sum]
                subserver_model.set_weights(subserver_weights_mean)
            # Create and open the CSV file for writing

            output = pd.DataFrame({"Avg":[np.mean(losses)]})
            output.to_csv(os.path.join("output", "avg_defense_"+str(DL_size_ratio)+".csv"), mode='a', index=False,header=False)
            # Model aggregation loop at the root
            root_weights = root_model.get_weights()
            root_weights_sum = [np.zeros_like(w) for w in root_weights]
            
            root_weights_sum= robust_aggregation_with_adaptive_thresholds_and_outlier_detection(subservers_models)
            root_weights_mean = [(w/ num_subservers) for w in root_weights_sum]
        
            root_model.set_weights(root_weights_mean)
            
    # Return the root model
    return root_model



# Example usage
num_subservers = 1
num_devices = 2
num_global_rounds = 1
num_local_rounds = 1
num_epochs_per_round = 2
learning_rate = 0.001

batch_size = 100  


In_set_file=loadmat('DLCB_dataset/O1_drone_200_I1_2p5/DLCB_input.mat')
Out_set_file=loadmat('DLCB_dataset/O1_drone_200_I1_2p5/DLCB_output.mat')

In_set=In_set_file['DL_input']
Out_set=Out_set_file['DL_output']


# Evaluate the final model
num_user_tot=In_set.shape[0]
n_DL_size=[.8, .9, 1]
#n_DL_size=[.15, .2, .25, .3, .35, .4, .45, .5, .55, .6, .7, .8, .9, 1]
count=0  
num_tot_TX=4 


for DL_size_ratio in n_DL_size:
    
    DL_size=int(num_user_tot*DL_size_ratio)
    count=count+1
    
    
    
    np.random.seed(2016)
    n_examples = DL_size
    num_train  = int(DL_size * 0.8)
    num_test   = int(num_user_tot*.2)
    
    train_index = np.random.choice(range(0,num_user_tot), size=num_train, replace=False)
    rem_index = set(range(0,num_user_tot))-set(train_index)
    test_index= list(set(np.random.choice(list(rem_index), size=num_test, replace=False)))
    
    In_train = In_set[train_index]
    In_test =  In_set[test_index] 
        
    Out_train = Out_set[train_index]
    Out_test = Out_set[test_index]
    
    
    
    in_shp = list(In_train.shape[1:])
    num_user_tot=In_set.shape[0]
    n_beams=128
    final_model = hierarchical_federated_learning(num_subservers, num_devices, num_global_rounds, num_local_rounds, num_epochs_per_round, learning_rate,n_beams,batch_size,In_train,In_test,Out_train,Out_test,DL_size_ratio)
          
    
    train_index = np.random.choice(range(0,num_user_tot), size=num_train, replace=False)
    rem_index = set(range(0,num_user_tot))-set(train_index)
    test_index= list(set(np.random.choice(list(rem_index), size=num_test, replace=False)))
    beams_predicted=final_model.predict( In_test, batch_size=batch_size, verbose=0)
    
    
    
    DL_Result={}
    for idx in range(0,num_devices,1): 
        beams_predicted=final_model.predict( In_test, batch_size=batch_size, verbose=0)
    
        DL_Result['TX'+str(idx+1)+'Pred_Beams']=beams_predicted
        DL_Result['TX'+str(idx+1)+'Opt_Beams']=Out_test[:,idx*n_beams:(idx+1)*n_beams]
    
    DL_Result['user_index']=test_index
    
    if not os.path.exists('./DLCB_code_output/defense'):
                          os.makedirs('DLCB_code_output/defense')
    savemat('DLCB_code_output/defense/DL_Result'+str(count)+'.mat',DL_Result)



In_train= 240
num_subservers= 1
num_devices= 2
number_sample_per_device= 120
global_round = 0
local_round = 0
device = 0
(125, 128)


KeyboardInterrupt: 